# RL for Autonomous Vehicle Tasks: Safety and Traffic Optimization

## Agents

In [2]:
# TODO: define agents here

## Scenarios

In [3]:
import gymnasium
import highway_env
from matplotlib import pyplot as plt
%matplotlib inline
from stable_baselines3 import DQN, PPO, A2C, SAC
from stable_baselines3.common.monitor import Monitor
import logging
import json
import os
import numpy as np
import tensorboard

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


### Car following

In [4]:
# Reward functions remain unchanged
def speed_matching_reward(v_ego, v_lead):
    return 1.0 - abs(v_ego - v_lead) / max(v_lead, 1.0)  # Normalize to [0, 1].

def safe_distance_penalty(d_rel, v_rel):
    desired_distance = 2.0  # Optimal distance in meters.
    safe_margin = 1.0       # Allowable range around desired distance.
    if d_rel < desired_distance - safe_margin:  # Too close, penalize heavily
        return -1.0
    elif d_rel > desired_distance + safe_margin:  # Too far, minor penalty
        return -0.2
    else:  # Reward for being at a safe following distance
        return 0.5

def optimal_speed_range_reward(v_ego, v_lead, v_min=10, v_max=15):
    if (v_ego < v_min or v_ego > v_max) and v_lead > v_ego: 
        return -0.5
    else:
        return 0.2

def reward_function(env, ego_vehicle, lead_vehicle):
    d_rel = lead_vehicle.position[0] - ego_vehicle.position[0]
    v_rel = lead_vehicle.velocity[0] - ego_vehicle.velocity[0]
    v_ego = ego_vehicle.velocity[0]
    v_lead = lead_vehicle.velocity[0]

    # Parameters
    desired_distance = 10.0  # Optimal distance in meters.
    safe_distance_margin = 5.0  # Allowable deviation from desired distance.
    optimal_speed = 20.0  # Optimal cruising speed in m/s.

    # Safe distance reward
    if d_rel < desired_distance - safe_distance_margin:
        safe_distance_reward = -1.0  # Penalty for being too close.
    elif d_rel > desired_distance + safe_distance_margin:
        safe_distance_reward = -0.5  # Penalty for being too far.
    else:
        safe_distance_reward = 1.0  # Reward for maintaining safe distance.

    # Speed matching reward
    speed_match_reward = 1.0 - abs(v_ego - v_lead) / max(v_lead, 1.0)

    # Optimal cruising speed bonus
    if abs(v_ego - optimal_speed) < 2.0:  # Close to optimal speed.
        cruising_speed_reward = 0.5
    else:
        cruising_speed_reward = -0.2

    # Collision penalty
    if ego_vehicle.crashed:
        collision_penalty = -10.0
    else:
        collision_penalty = 0.0

    # Reward for reaching target distance safely
    target_distance = env.road.network.get_lane(ego_vehicle.lane_index).length
    if ego_vehicle.position[0] > target_distance:
        target_distance_reward = 10.0
    else:
        target_distance_reward = 0.0

    # Combine rewards
    total_reward = (
        safe_distance_reward +
        speed_match_reward +
        cruising_speed_reward +
        collision_penalty +
        target_distance_reward
    )

    return total_reward


# Discrete action space environment
car_following_discrete = gymnasium.make('highway-v0', render_mode="rgb_array")
car_following_discrete.unwrapped.configure({
    "vehicles_count": 2,
    "controlled_vehicles": 1,
    "lanes_count": 1,
    "action": {
        "type": "DiscreteMetaAction"  # Uses a discrete action space supported by your algorithm.
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 20,
        "features": ["x", "y", "vx", "vy"],
        "absolute": False,
        "normalize": True
    },
    "screen_width": 600,             
    "screen_height": 600,
    "duration": 100,                  
    "offscreen_rendering": True, 
    "show_trajectories": True
})
car_following_discrete.reset()
car_following_discrete.reward_function = lambda obs, action: reward_function(
    car_following_discrete, car_following_discrete.vehicle, car_following_discrete.road.vehicles[1]
)

# Continuous action space environment
car_following_continuous = gymnasium.make('highway-v0', render_mode="rgb_array")
car_following_continuous.unwrapped.configure({
    "vehicles_count": 2,
    "controlled_vehicles": 1,
    "lanes_count": 1,
    "action": {
        "type": "ContinuousAction"  # Define continuous actions for smooth control.
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 20,
        "features": ["x", "y", "vx", "vy"],
        "absolute": False,
        "normalize": True
    },
    "screen_width": 600,             
    "screen_height": 600,
    "duration": 100,                  
    "offscreen_rendering": True, 
    "show_trajectories": True
})
car_following_continuous.reset()
car_following_continuous.reward_function = lambda obs, action: reward_function(
    car_following_continuous, car_following_continuous.vehicle, car_following_continuous.road.vehicles[1]
)

### Highway cruising

In [5]:
def overtaking_reward_function(env, ego_vehicle, road_vehicles):
    d_rel = road_vehicles[0].position[0] - ego_vehicle.position[0]
    v_rel = road_vehicles[0].velocity[0] - ego_vehicle.velocity[0]
    ego_lane = ego_vehicle.lane_index

    # Reward for maintaining safe distance
    safe_distance_reward = safe_distance_penalty(d_rel, v_rel)
    
    # Reward for matching speed
    speed_match_reward = speed_matching_reward(ego_vehicle.velocity[0], road_vehicles[0].velocity[0])

    # Reward for staying within the optimal speed range (when not overtaking)
    optimal_speed_range_reward = optimal_speed_range_reward(ego_vehicle.velocity[0], road_vehicles[0].velocity[0])

    # Overtaking specific reward: Reward when agent overtakes and does it safely
    overtaking_reward = 0
    if ego_vehicle.position[0] > road_vehicles[0].position[0] and ego_vehicle.lane_index != ego_lane:
        overtaking_reward = 5  # Reward for overtaking successfully

    # Reward for initiating the overtaking maneuver (e.g., changing lanes)
    if ego_vehicle.position[0] < road_vehicles[0].position[0] and ego_vehicle.lane_index != ego_lane:
        overtaking_initiation_reward = 2  # Reward for starting overtaking maneuver
    else:
        overtaking_initiation_reward = 0

    # **Reward for staying in the correct lane after overtaking:**
    correct_lane_reward = 0
    if ego_vehicle.lane_index == 1:  # Assuming lane index 1 is the right lane
        correct_lane_reward = 2  # Reward if staying in the right lane after overtaking
    
    # **Penalty for going into the wrong lane (unnecessary lane changes after overtaking):**
    incorrect_lane_penalty = 0
    if ego_vehicle.lane_index != 1 and ego_vehicle.position[0] > road_vehicles[0].position[0]:  # When in wrong lane after overtaking
        incorrect_lane_penalty = -2  # Small penalty for drifting into the wrong lane

    # Penalize for collisions during overtaking
    if ego_vehicle.crashed:
        return -10.0

    # Reward the agent for completing a successful overtake without crashing
    if ego_vehicle.position[0] > env.road.network.get_lane(ego_vehicle.lane_index).length:
        return 10.0

    # Combine all rewards
    total_reward = (safe_distance_reward + 
                    speed_match_reward + 
                    optimal_speed_range_reward + 
                    overtaking_reward + 
                    correct_lane_reward + 
                    incorrect_lane_penalty + 
                    overtaking_initiation_reward)

    return total_reward


# Environment for Discrete Actions (DQN, A2C)
highway_cruising_discrete = gymnasium.make('highway-v0', render_mode="rgb_array")
highway_cruising_discrete.unwrapped.configure({
    "vehicles_count": 50,  # Increased vehicle count to make the traffic denser.
    "controlled_vehicles": 1,  # One controlled vehicle (ego vehicle).
    "lanes_count": 3,  # More lanes to test the agent's ability to overtake in traffic.
    "road_type": "straight",  # Use straight road to make traffic management simpler (but still complex).
    "vehicles_density": 1.0,  # Higher density for more vehicles.
    "action": {
        "type": "DiscreteMetaAction"  # Discrete action space suitable for DQN and A2C
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 20,  # Including traffic vehicles.
        "features": ["x", "y", "vx", "vy"],  # Key kinematic features.
        "absolute": False,
        "normalize": True  # Normalize observations.
    },
    "screen_width": 800,  # Larger screen for a wider view of the environment.
    "screen_height": 800,  # Larger height for better vehicle representation.
    "duration": 500,  # Increase the duration to simulate longer driving periods.
    "offscreen_rendering": True,
    "show_trajectories": True,  # Keep track of trajectories to see pathing.
    "traffic_density": 1.5,  # Increased traffic density for better generalization.
    "highway_length": 4000  # Make the highway longer to simulate more complex navigation.
})

# Reset environment
highway_cruising_discrete.reset()

# Assign the reward function
highway_cruising_discrete.reward_function = lambda obs, action: overtaking_reward_function(
    highway_cruising_discrete, highway_cruising_discrete.vehicle, highway_cruising_discrete.road.vehicles
)


# Continuous action space - Configuration for PPO and SAC
highway_cruising_continuous = gymnasium.make('highway-v0', render_mode="rgb_array")
highway_cruising_continuous.unwrapped.configure({
    "vehicles_count": 50,  # Increased vehicle count to make the traffic denser.
    "controlled_vehicles": 1,  # One controlled vehicle (ego vehicle).
    "lanes_count": 3,  # More lanes to test the agent's ability to overtake in traffic.
    "road_type": "straight",  # Use straight road to make traffic management simpler (but still complex).
    "vehicles_density": 1.0,  # Higher density for more vehicles.
    "action": {
        "type": "ContinuousAction"  # Continuous action space for PPO and SAC
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 20,  # Including traffic vehicles.
        "features": ["x", "y", "vx", "vy"],  # Key kinematic features.
        "absolute": False,
        "normalize": True  # Normalize observations.
    },
    "screen_width": 800,  # Larger screen for a wider view of the environment.
    "screen_height": 800,  # Larger height for better vehicle representation.
    "duration": 500,  # Increase the duration to simulate longer driving periods.
    "offscreen_rendering": True,
    "show_trajectories": True,  # Keep track of trajectories to see pathing.
    "traffic_density": 1.5,  # Increased traffic density for better generalization.
    "highway_length": 4000  # Make the highway longer to simulate more complex navigation.
})

# Reset environment
highway_cruising_continuous.reset()

# Assign the reward function
highway_cruising_continuous.reward_function = lambda obs, action: overtaking_reward_function(
    highway_cruising_continuous, highway_cruising_continuous.vehicle, highway_cruising_continuous.road.vehicles
)

### Roundabout (2nd version)

In [6]:
def roundabout_reward_function(env, ego_vehicle, road_vehicles):
    # Rewards and penalties
    merging_reward = 0
    exiting_reward = 0
    collision_penalty = 0
    yielding_reward = 0
    smoothness_penalty = 0
    wrong_way_penalty = 0
    optimal_speed_bonus = 0
    intermediate_reward = 0  # New: reward for intermediate progress in actions

    # Determine ego-vehicle states
    is_in_roundabout = ego_vehicle.lane_index == env.roundabout_lane_index
    is_exiting_roundabout = ego_vehicle.is_exiting_roundabout()  # Exiting logic needed
    is_yielding = ego_vehicle.is_yielding()
    speed = ego_vehicle.velocity[0]

    # Reward for merging successfully into the roundabout
    if is_in_roundabout and not ego_vehicle.just_entered_roundabout():
        merging_reward = 5  # Encourage entering roundabout correctly

    # Reward for exiting roundabout successfully
    if is_exiting_roundabout:
        exiting_reward = 10  # High reward for completing the roundabout navigation

    # Penalty for collision
    if ego_vehicle.crashed:
        collision_penalty = -10.0  # Strongly discourage collisions

    # Yielding appropriately (optional reward)
    if is_yielding:
        yielding_reward = 2  # Small reward for good yielding behavior

    # Penalize stalling or stopping unnecessarily
    if speed < 0.1 and not is_yielding:
        smoothness_penalty = -2  # Penalize unnecessary stopping

    # Penalize wrong-way navigation
    if not ego_vehicle.is_in_correct_lane():
        wrong_way_penalty = -5  # Penalize entering/exiting via wrong lanes

    # Bonus for maintaining a safe speed inside roundabout
    if is_in_roundabout:
        if env.config["reward_speed_range"][0] <= speed <= env.config["reward_speed_range"][1]:
            optimal_speed_bonus = 1.0  # Encourage navigating at appropriate speeds

    # Intermediate reward for progress (for continuous actions, we can reward gradual approach)
    if is_in_roundabout and not is_exiting_roundabout:
        intermediate_reward = 1.0  # Reward for making progress in the roundabout

    # Combine all reward components
    total_reward = (
        merging_reward +
        exiting_reward +
        yielding_reward +
        optimal_speed_bonus +
        collision_penalty +
        smoothness_penalty +
        wrong_way_penalty +
        intermediate_reward
    )

    return total_reward


# Configure roundabout environment for both discrete and continuous actions
def make_roundabout_env(action_type='Discrete'):
    roundabout_env = gymnasium.make('roundabout-v0', render_mode="rgb_array")
    
    if action_type == 'Discrete':
        roundabout_env.unwrapped.configure({
            "vehicles_count": 20,                 # Include moderate traffic
            "controlled_vehicles": 1,            # Single ego vehicle
            "lanes_count": 1,                    # Simulate a single-lane roundabout
            "action": {
                "type": "DiscreteMetaAction"     # For discrete action space
            },
            "observation": {
                "type": "Kinematics",            # Track positions, velocities, etc.
                "vehicles_count": 20,
                "features": ["x", "y", "vx", "vy"],
                "absolute": False,
                "normalize": True
            },
            "screen_width": 800,                 # Increase resolution for visual clarity
            "screen_height": 800,
            "road_width": 20,                    # Adjust for a roundabout layout
            "duration": 10,                      # Increase episode duration
            "offscreen_rendering": True,
            "roundabout": {
                "radius": 30,                    # Roundabout radius
                "entry_exit_points": 4,          # Four entry/exit points (North, South, East, West)
                "entry_lanes": 1,                # Two lanes leading into the roundabout
                "exit_lanes": 1                  # Two lanes exiting the roundabout
            },
            "collision_reward": -10.0,           # Heavier penalty for collisions
            "roundabout_yielding": True,         # Vehicles give way properly in roundabout
            "reward_speed_range": [5, 20],      # Optimal speeds for navigating
            "normalize_observations": True,      # Normalize observations for better training
            "show_trajectories": True            # Show trajectories for visualization
        })
    
    elif action_type == 'Continuous':
        roundabout_env.unwrapped.configure({
            "vehicles_count": 20,                 # Include moderate traffic
            "controlled_vehicles": 1,            # Single ego vehicle
            "lanes_count": 1,                    # Simulate a single-lane roundabout
            "action": {
                "type": "ContinuousMetaAction"   # For continuous action space
            },
            "observation": {
                "type": "Kinematics",            # Track positions, velocities, etc.
                "vehicles_count": 20,
                "features": ["x", "y", "vx", "vy"],
                "absolute": False,
                "normalize": True
            },
            "screen_width": 800,                 # Increase resolution for visual clarity
            "screen_height": 800,
            "road_width": 20,                    # Adjust for a roundabout layout
            "duration": 10,                      # Increase episode duration
            "offscreen_rendering": True,
            "roundabout": {
                "radius": 30,                    # Roundabout radius
                "entry_exit_points": 4,          # Four entry/exit points (North, South, East, West)
                "entry_lanes": 1,                # Two lanes leading into the roundabout
                "exit_lanes": 1                  # Two lanes exiting the roundabout
            },
            "collision_reward": -10.0,           # Heavier penalty for collisions
            "roundabout_yielding": True,         # Vehicles give way properly in roundabout
            "reward_speed_range": [5, 20],      # Optimal speeds for navigating
            "normalize_observations": True,      # Normalize observations for better training
            "show_trajectories": True            # Show trajectories for visualization
        })
    
    return roundabout_env

# Example for using both types of environments
roundabout_discrete_env = make_roundabout_env('Discrete')
roundabout_continuous_env = make_roundabout_env('Continuous')

# Assign the reward function
roundabout_discrete_env.reward_function = lambda obs, action: roundabout_reward_function(
    roundabout_discrete_env, roundabout_discrete_env.vehicle, roundabout_discrete_env.road.vehicles[1:]
)

roundabout_continuous_env.reward_function = lambda obs, action: roundabout_reward_function(
    roundabout_continuous_env, roundabout_continuous_env.vehicle, roundabout_continuous_env.road.vehicles[1:]
)

## Training and Testing

In [7]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def create_directory(path):
    """
    Create directory if it doesn't exist
    
    Args:
        path (str): Directory path to create
    """
    try:
        os.makedirs(path, exist_ok=True)
        logger.info(f"Directory created: {path}")
    except Exception as e:
        logger.error(f"Error creating directory {path}: {e}")

def save_pipeline_data(agent_name, stage, environment_name, data):
    """
    Save pipeline stage data
    
    Args:
        agent_name (str): Name of the RL agent
        stage (str): Current pipeline stage
        environment_name (str): Name of the environment
        data (dict): Data to be saved
    """
    create_directory(f"results/{agent_name}")
    
    try:
        filename = f"results/{agent_name}/{environment_name}_{stage}_data.json"
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
        logger.info(f"Data saved for {stage} in {environment_name}")
    except Exception as e:
        logger.error(f"Error saving data: {e}")

In [8]:
def train(agent, environment, agent_name, stage, environment_name, timesteps=1000):
    """
    Train the agent in a specific environment and save the model
    
    Args:
        agent (sb3.BaseAlgorithm): RL agent to train
        environment (gym.Env): Environment to train in
        agent_name (str): Name of the agent
        stage (str): Current pipeline stage
        environment_name (str): Name of the environment
        timesteps (int, optional): Number of training timesteps. Defaults to 100.
    
    Returns:
        Trained agent
    """
    try:
        # Wrap environment with Monitor
        log_dir = f"logs/{agent_name}"
        os.makedirs(log_dir, exist_ok=True)
        env = Monitor(environment, log_dir)
            
        # Set environment and learn
        agent.set_env(env)
        agent.learn(total_timesteps=timesteps)

        # Get training results
        training_results = {
            'rewards': env.get_episode_rewards(),
        }
        
        # Create directories for model and results
        create_directory(f"models/{agent_name}")
        
        # Save model
        agent.save(f"models/{agent_name}/{environment_name}_model")
        
        # Prepare and save training information
        training_info = {
            'agent_name': agent_name,
            'environment_name': environment_name,
            'stage': stage,
            'total_timesteps': timesteps,
            'training_results': training_results
        }
        save_pipeline_data(agent_name, "train", environment_name, training_info)
        
        logger.info(f"Training completed for {agent_name} in {environment_name}")
        return agent
    
    except Exception as e:
        logger.error(f"Training failed for {agent_name} in {environment_name}: {e}")
        raise



In [9]:
def test(agent, environment, agent_name, stage, environment_name, num_episodes=10):
    test_results = []
    total_rewards = []
    total_collisions = 0
    traffic_speeds = []

    for episode in range(num_episodes):
        try:
            obs, info = environment.reset()
            done = truncated = False
            episode_reward = 0
            episode_collisions = 0
            episode_speeds = []

            while not (done or truncated):
                # Predict action using the agent
                action, _states = agent.predict(obs, deterministic=True)
                obs, reward, done, truncated, info = environment.step(action)

                # Update reward
                episode_reward += reward
                
                # Check for collisions
                if info.get("crashed", False):
                    episode_collisions += 1

                # Gather traffic speeds
                current_speeds = [
                    vehicle.speed
                    for vehicle in environment.unwrapped.road.vehicles  # Use unwrapped
                    if vehicle != environment.unwrapped.vehicle  # Exclude agent vehicle
                ]
                episode_speeds.extend(current_speeds)

                # Optional rendering
                environment.render()

            environment.close()

            # Collect data for this episode
            total_rewards.append(episode_reward)
            total_collisions += episode_collisions
            traffic_speeds.extend(episode_speeds)

            test_results.append({
                'episode': episode,
                'total_reward': episode_reward,
                'collisions': episode_collisions,
                'avg_speed': np.mean(episode_speeds) if episode_speeds else 0,
                'speed_variance': np.var(episode_speeds) if episode_speeds else 0
            })
        
        except Exception as e:
            logger.error(f"Test episode {episode + 1} failed in {environment_name}: {e}")

    # Calculate overall KPIs
    avg_reward = np.mean(total_rewards)
    avg_speed = np.mean(traffic_speeds) if traffic_speeds else 0
    speed_variance = np.var(traffic_speeds) if traffic_speeds else 0

    logger.info(f"Test completed in {environment_name} - "
                f"Avg Reward: {avg_reward}, Total Collisions: {total_collisions}, "
                f"Avg Traffic Speed: {avg_speed}, Speed Variance: {speed_variance}")

    # Save test results and KPIs
    save_pipeline_data(agent_name, stage, environment_name, {
        'test_results': test_results,
        'kpis': {
            'average_reward': avg_reward,
            'total_collisions': total_collisions,
            'average_speed': avg_speed,
            'speed_variance': speed_variance
        },
        'environment': environment_name,
        'stage': stage
    })

    return {
        'test_results': test_results,
        'kpis': {
            'average_reward': avg_reward,
            'total_collisions': total_collisions,
            'average_speed': avg_speed,
            'speed_variance': speed_variance
        }
    }

In [10]:
def rl_pipeline(initial_agent, agent_name, environments, timesteps=1000):
    """
    Sequential environment training pipeline with transfer learning considerations
    
    Args:
        initial_agent (sb3.BaseAlgorithm): Initial RL agent
        agent_name (str): Name of the agent
        environments (list of tuples): A list of (env_name, gym.Env) tuples defining the order of execution.
            Example: [("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)]
    
    Returns:
        Trained agent
    """

    logger.info("Customizable Sequential Environment Pipeline with Transfer Learning Started")
    
    current_agent = initial_agent

    for i, (env_name, env) in enumerate(environments):
        logger.info(f"Stage {i + 1}: {env_name.capitalize()} Environment")
        
        # Test previous agent in current environment
        if i > 0:
            previous_env_name = environments[i - 1][0]
            logger.info(f"Testing {previous_env_name.capitalize()} Agent in {env_name.capitalize()}")
            test(current_agent, env, agent_name, 'pre_training_test', env_name)
        
        # Fine-tune the agent from previous environment if applicable (i.e., transfer learning)
        logger.info(f"Training in {env_name.capitalize()} Environment")
        current_agent = train(
            current_agent, env, 
            agent_name, env_name, env_name, timesteps=timesteps
        )
        
        # Test the agent's performance after training in the current environment
        logger.info(f"Testing in {env_name.capitalize()} Environment")
        test(current_agent, env, agent_name, 'post_training_test', env_name)
        
        # Optionally, save a checkpoint after each environment
        if i % 5 == 0:  # For example, save every 5 environments
            checkpoint_dir = f"models/{agent_name}/checkpoint_{i}"
            create_directory(checkpoint_dir)
            current_agent.save(checkpoint_dir)

    logger.info("Customizable Sequential Environment Pipeline Completed")

    # Save the final agent after all environments
    create_directory(f"models/{agent_name}/final")
    current_agent.save(f"models/{agent_name}/{agent_name}_final_model")
    
    return current_agent

In [11]:
# %tensorboard --logdir "agent_DQN"

In [12]:
from gymnasium import spaces
import numpy as np

class NormalizeObservation(gymnasium.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,4), dtype=np.float32)

    def observation(self, obs):
        # Normalize or transform the observations here
        normalized_obs = obs[:4]  # Take only the first 4 features or adjust as needed
        return normalized_obs

# Wrap environments
car_following_discrete = NormalizeObservation(car_following_discrete)
car_following_continuous = NormalizeObservation(car_following_continuous)
highway_cruising_discrete = NormalizeObservation(highway_cruising_discrete)
highway_cruising_continuous = NormalizeObservation(highway_cruising_continuous)
roundabout_discrete_env = NormalizeObservation(roundabout_discrete_env)
roundabout_continuous_env = NormalizeObservation(roundabout_continuous_env)

## Discrete action space models

### Agent DQN

In [13]:
agent_DQN = DQN('MlpPolicy', roundabout_discrete_env,
        policy_kwargs=dict(net_arch=[256, 256]),
        learning_rate = 1e-4,
        buffer_size=15000,
        learning_starts=200,
        batch_size=32,
        gamma=0.8,
        train_freq=1,
        gradient_steps=1,
        target_update_interval=50,
        verbose=1,
        tensorboard_log="agent_DQN/", 
        device='auto')

final_agent_DQN = rl_pipeline(agent_DQN, 'agent_DQN', [("car_following", car_following_discrete), ("highway_crusing", highway_cruising_discrete), ("roundabout", roundabout_discrete_env)], 100)

2025-01-01 12:06:08,941 - INFO - Customizable Sequential Environment Pipeline with Transfer Learning Started
2025-01-01 12:06:08,942 - INFO - Stage 1: Car_following Environment
2025-01-01 12:06:08,942 - INFO - Training in Car_following Environment


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a DummyVecEnv.
Logging to agent_DQN/DQN_63
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.75     |
|    ep_rew_mean      | 3.76     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 134      |
|    time_elapsed     | 0        |
|    total_timesteps  | 23       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.5      |
|    ep_rew_mean      | 5.66     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 145      |
|    time_elapsed     | 0        |
|    total_timesteps  | 68       |
----------------------------------
----------------------------------
| rollout/            |          |
|  

2025-01-01 12:06:09,631 - INFO - Directory created: models/agent_DQN
2025-01-01 12:06:09,634 - INFO - Directory created: results/agent_DQN
2025-01-01 12:06:09,635 - INFO - Data saved for train in car_following
2025-01-01 12:06:09,635 - INFO - Training completed for agent_DQN in car_following
2025-01-01 12:06:09,635 - INFO - Testing in Car_following Environment
2025-01-01 12:06:11,427 - INFO - Test completed in car_following - Avg Reward: 2.903111111111111, Total Collisions: 10, Avg Traffic Speed: 19.49766154878366, Speed Variance: 10.698451310581486
2025-01-01 12:06:11,427 - INFO - Directory created: results/agent_DQN
2025-01-01 12:06:11,428 - INFO - Data saved for post_training_test in car_following
2025-01-01 12:06:11,428 - INFO - Directory created: models/agent_DQN/checkpoint_0
2025-01-01 12:06:11,431 - INFO - Stage 2: Highway_crusing Environment
2025-01-01 12:06:11,432 - INFO - Testing Car_following Agent in Highway_crusing


### Agent A2C

In [116]:
agent_A2C = A2C('MlpPolicy', roundabout_discrete_env,
        policy_kwargs=dict(net_arch=[256, 256, 128]),  # Slightly deeper network
        learning_rate=3e-4,  # Higher learning rate for faster updates
        n_steps=128,  # Longer trajectory for gradient estimation
        gamma=0.99,
        gae_lambda=1.0,  # GAE with full weighting of future advantages
        ent_coef=0.01,  # Encourage exploration
        vf_coef=0.5,  # Value function loss coefficient
        max_grad_norm=0.5,  # Gradient clipping for stability
        tensorboard_log="agent_A2C/")


final_agent_A2C = rl_pipeline(agent_A2C, 'agent_A2C', [("car_following", car_following_discrete), ("highway_crusing", highway_cruising_discrete), ("roundabout", roundabout_discrete_env)], 200000)

2024-12-31 18:13:29,676 - INFO - Customizable Sequential Environment Pipeline with Transfer Learning Started
2024-12-31 18:13:29,676 - INFO - Stage 1: Car_following Environment
2024-12-31 18:13:29,677 - INFO - Training in Car_following Environment
2024-12-31 18:13:29,677 - ERROR - Training failed for agent_A2C in car_following: Observation spaces do not match: Box(-inf, inf, (5, 5), float32) != Box(-inf, inf, (20, 4), float32)


ValueError: Observation spaces do not match: Box(-inf, inf, (5, 5), float32) != Box(-inf, inf, (20, 4), float32)

## Continuous action space models

### Agent PPO

In [117]:
agent_PPO = PPO('MlpPolicy', roundabout_continuous_env,
        policy_kwargs=dict(net_arch=[256, 256, 128]),  # Deeper network for discrete actions
        learning_rate=3e-4,  # Slightly higher for faster convergence
        n_steps=4096,  # Larger trajectory length for diverse experiences
        batch_size=128,  # Larger batch size for more stable updates
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        clip_range_vf=None,  # Default behavior
        ent_coef=0.01,  # Encourage exploration
        vf_coef=0.5,
        max_grad_norm=0.5,
        tensorboard_log="agent_PPO_discrete/")


final_agent_PPO = rl_pipeline(agent_PPO, 'agent_PPO', [("car_following", car_following_continuous), ("highway_crusing", highway_cruising_continuous), ("roundabout", roundabout_continuous_env)], 1000)

2024-12-31 18:13:34,127 - INFO - Customizable Sequential Environment Pipeline with Transfer Learning Started
2024-12-31 18:13:34,127 - INFO - Stage 1: Car_following Environment
2024-12-31 18:13:34,127 - INFO - Training in Car_following Environment
2024-12-31 18:13:34,128 - ERROR - Training failed for agent_PPO in car_following: Observation spaces do not match: Box(-inf, inf, (5, 5), float32) != Box(-inf, inf, (20, 4), float32)


ValueError: Observation spaces do not match: Box(-inf, inf, (5, 5), float32) != Box(-inf, inf, (20, 4), float32)

### Agent SAC

In [72]:
agent_SAC = SAC(
    'MlpPolicy', 
    roundabout_continuous_env,
    policy_kwargs=dict(net_arch=[256, 256]),  # Adjust neural network architecture
    learning_rate=1e-4,  # Default learning rate, can try 3e-4 or 1e-5 for more stability
    buffer_size=50000,  # Increase buffer size to better store experience, especially for larger tasks
    learning_starts=1000,  # Start learning after 1000 steps for better exploration
    batch_size=64,  # Larger batch size often improves performance
    gamma=0.99,  # Higher discount factor helps agent look further into the future (good for more complex environments)
    train_freq=1,  # Frequency of model updates (every step)
    gradient_steps=1,  # Gradient steps per update (increase to improve stability)
    target_update_interval=1,  # Update target network after each step for quicker adaptation
    tau=0.005,  # Soft update for the target network (smaller tau = slower update)
    use_sde=True,  # Use Stochastic Differential Equations for better exploration
    tensorboard_log="agent_SAC/",  # Log for TensorBoard visualization
    verbose=1  # Increase verbosity for debugging and monitoring
)

final_agent_SAC = rl_pipeline(agent_SAC, 'agent_SAC', [("car_following", car_following_continuous), ("highway_crusing", highway_cruising_continuous), ("roundabout", roundabout_continuous_env)])#("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)])

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.box.Box'>,) as action spaces but Discrete(5) was provided

## Performance

In [73]:
# Display results
def display_results(agent_name, stages, type="table"):
    # Load results
    results = {}
    for stage in stages:
        with open(f"results/{agent_name}/{stage}_data.json", 'r') as f:
            results[stage] = json.load(f)

    if type == "list":
        print(f"--- {agent_name} ---\n")
        for stage, data in results.items():
            print(f"> {stage}")
            print(f"    - Average Reward: {data['kpis']['average_reward']}")
            print(f"    - Total Collisions: {data['kpis']['total_collisions']}")
            print(f"    - Average Speed: {data['kpis']['average_speed']}")
            print(f"    - Speed Variance: {data['kpis']['speed_variance']}")
            print("")
        print("--------------------\n")
    elif type == "table":
        table = []
        for stage, data in results.items():
            table.append([stage, data['kpis']['average_reward'], data['kpis']['total_collisions'], data['kpis']['average_speed'], data['kpis']['speed_variance']])
        table = np.array(table)

        print("|".join([f"{{[{str(agent_name)}]:<20}} {str('Stage'):>10} ", "Average Reward".center(20), "Total Collisions".center(20), "Average Speed".center(20), "Speed Variance".center(20)]))
        print("=" * 117)
        for row in table:
            formatted_row = [
                f"{str(row[0]):>31} ",    # Stage (left aligned)
                f"{str(row[1]):^20}",    # Average Reward (center aligned)
                f"{str(row[2]):^20}",    # Total Collisions (center aligned)
                f"{str(row[3]):^20}",    # Average Speed (center aligned)
                f"{str(row[4]):^20}",     # Speed Variance (center aligned)
            ]
            print("|".join(formatted_row))
            print("-" * 117)
        print()
        

In [74]:
display_results("agent_DQN", ["lane_changing_pos_training_test", "roundabout_pre_training_test", "roundabout_pos_training_test", "overtaking_pre_training_test", "overtaking_pos_training_test"])

{[agent_DQN]:<20}      Stage |   Average Reward   |  Total Collisions  |   Average Speed    |   Speed Variance   
lane_changing_pos_training_test | 19.809523809523807 |         0          | 21.490179070718053 | 2.5431416061097925 
---------------------------------------------------------------------------------------------------------------------
   roundabout_pre_training_test | 4.444117647058824  |         6          | 15.240398142143176 |  5.1120777379011   
---------------------------------------------------------------------------------------------------------------------
   roundabout_pos_training_test | 6.932352941176473  |         3          | 15.098922704992187 | 9.354996121165309  
---------------------------------------------------------------------------------------------------------------------
   overtaking_pre_training_test | 9.662684444444448  |         8          | 20.412763661430365 | 2.3286179404918492 
----------------------------------------------------------------

In [83]:
# Test the final agent in all environments
def test_agent_all_envs(agent_name, environments, environment_names):
    # Load the final agent
    #agent = DQN.load(f"models/{agent_name}/{agent_name}_final_model", device="cpu")
    agent = A2C.load(f"models/{agent_name}/{environment_names[0]}_model", device="cpu")
    # Lane Changing
    performance_lane_changing = test(agent, environments[0], agent_name, '', "final_test_" + environment_names[0])

    # Roundabout
    performance_roundabout = test(agent, environments[1], agent_name, '', "final_test_" + environment_names[1])

    # Overtaking
    performance_overtaking = test(agent, environments[2], agent_name, '', "final_test_" + environment_names[2])


In [86]:
test_agent_all_envs("agent_DQN", [car_following_discrete], ["car_following"])#[lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])


# test_agent_all_envs("agent_DQN_rol", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_olr", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_orl", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_rlo", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_lor", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])


2024-12-31 15:29:11,256 - INFO - Test completed in final_test_car_following - Avg Reward: 62.074447933497595, Total Collisions: 2, Avg Traffic Speed: 21.609681835097486, Speed Variance: 1.057735135632107
2024-12-31 15:29:11,257 - INFO - Directory created: results/agent_A2C
2024-12-31 15:29:11,257 - INFO - Data saved for  in final_test_car_following


IndexError: list index out of range

## Analysis

In [20]:
# Display final test results
display_results("agent_DQN", ["final_test_lane_changing_", "final_test_roundabout_", "final_test_overtaking_"])

{[agent_DQN]:<20}      Stage |   Average Reward   |  Total Collisions  |   Average Speed    |   Speed Variance   
      final_test_lane_changing_ |      16.4125       |         4          | 20.602228832201057 | 1.7628795927127507 
---------------------------------------------------------------------------------------------------------------------
         final_test_roundabout_ |       12.05        |         0          | 14.381887780369855 | 19.82951714419477  
---------------------------------------------------------------------------------------------------------------------
         final_test_overtaking_ | 19.708888888888897 |         3          | 20.647436386620896 | 1.9794265486352098 
---------------------------------------------------------------------------------------------------------------------



## Conclusion